In [1]:
#!/usr/bin/env python
"""
Simple script to read and display TikTok data from parquet files.
"""

import pandas as pd
from pathlib import Path
import os
import sys

In [10]:
OUT_DIR = Path("../data/tiktok_data")
print(OUT_DIR)
parquet_files = list(OUT_DIR.glob("*.parquet"))

if not parquet_files:
    print("No parquet files found in", OUT_DIR)
    exit()

# Sort by modification time (newest first)
parquet_files = sorted(parquet_files, key=os.path.getmtime, reverse=True)
print(parquet_files)
# Allow selecting a specific file from command line
if len(sys.argv) > 1 and sys.argv[1].endswith('.parquet'):
    target_file = Path(sys.argv[1])
    if target_file in parquet_files:
        selected_file = target_file
    else:
        print(f"File {sys.argv[1]} not found. Using most recent file instead.")
        selected_file = parquet_files[0]
else:
    # Use the most recent file by default
    selected_file = parquet_files[0]

print(f"Reading {selected_file}")

# Load the data
df = pd.read_parquet(selected_file)

../data/tiktok_data
[PosixPath('../data/tiktok_data/tiktok_data.parquet')]
Reading ../data/tiktok_data/tiktok_data.parquet


In [11]:
df.head()

,video_id,posted_ts,description,author_id,author_name,follower_count,view_count,like_count,share_count,comment_count,repost_count,thumbnail_path,top_comments
0,7485152103380389127,1.742773e+09,Bubble took every impurities🫧 #skincare #korea...,7389943393130562577,heylina2484,99800,11383654,816847,42736,2857,0,../../tiktok_data/thumbnails/74851521033803891...,"[i thought it was mold..., ITU PASTI GA READY ..."
1,7478688849296772359,1.741268e+09,Bobacream… plus PDRN… and the result? 🧋➕🐟 It’s...,7288964931759457281,mintfactory1995,9474,1743649,76443,1821,388,0,../../tiktok_data/thumbnails/74786888492967723...,"[POV: cewek natural yang cowok maksud..🗿, If I..."
2,7483154654604397842,1.742308e+09,#skincare #skintok #kbeauty #koreanskincare #o...,7451563120218276880,malangcutiee,2592,2635129,153172,8892,972,0,../../tiktok_data/thumbnails/74831546546043978...,"[Для подростковой кожи подойдет?, skin1004??, ..."
3,7485337053383625991,1.742816e+09,My breakout routine pt.2 #acneproneskin #skinc...,7411074968203953169,frekkie59,8022,319306,47786,5522,48,0,../../tiktok_data/thumbnails/74853370533836259...,"[is it really good... I'm scared, So close! Ur..."
4,7497508042016115976,1.745650e+09,cleansing @skin1004 cleanser @celimax serum @l...,7380385709847036929,cami_kana,107200,2920533,260270,13394,1895,0,../../tiktok_data/thumbnails/74975080420161159...,"[kulit mukanyaa impiann bangett, pengaruh skin..."


In [16]:
# Find rows where thumbnail path doesn't exist and add "../" prefix
mask = df['thumbnail_path'].apply(lambda x: not Path(x).exists())
df.loc[mask, 'thumbnail_path'] = df.loc[mask, 'thumbnail_path'].apply(lambda x: x.replace('../..', '../data'))
df.loc[mask]

,video_id,posted_ts,description,author_id,author_name,follower_count,view_count,like_count,share_count,comment_count,repost_count,thumbnail_path,top_comments


In [15]:
#df.loc[df['top_comments'].str.len() == 0]
df.loc[df['thumbnail_path']==""]

,video_id,posted_ts,description,author_id,author_name,follower_count,view_count,like_count,share_count,comment_count,repost_count,thumbnail_path,top_comments


In [11]:
len(df.loc[df['top_comments'].str.len() == 0])

906

In [29]:
df['thumbnail_path'] = df['thumbnail_path'].apply(lambda x: x.replace('../', ''))

In [13]:
len(df)

10712

In [12]:
# Check for duplicates
print("Number of duplicate video_ids:", df['video_id'].duplicated().sum())

# Drop duplicates based on video_id
#df = df.drop_duplicates(subset=['video_id'])

# Show remaining data
df.head()

Number of duplicate video_ids: 0


,video_id,posted_ts,description,author_id,author_name,follower_count,view_count,like_count,share_count,comment_count,repost_count,thumbnail_path,top_comments
0,7485152103380389127,1.742773e+09,Bubble took every impurities🫧 #skincare #korea...,7389943393130562577,heylina2484,99800,11383654,816847,42736,2857,0,../../tiktok_data/thumbnails/74851521033803891...,"[i thought it was mold..., ITU PASTI GA READY ..."
1,7478688849296772359,1.741268e+09,Bobacream… plus PDRN… and the result? 🧋➕🐟 It’s...,7288964931759457281,mintfactory1995,9474,1743649,76443,1821,388,0,../../tiktok_data/thumbnails/74786888492967723...,"[POV: cewek natural yang cowok maksud..🗿, If I..."
2,7483154654604397842,1.742308e+09,#skincare #skintok #kbeauty #koreanskincare #o...,7451563120218276880,malangcutiee,2592,2635129,153172,8892,972,0,../../tiktok_data/thumbnails/74831546546043978...,"[Для подростковой кожи подойдет?, skin1004??, ..."
3,7485337053383625991,1.742816e+09,My breakout routine pt.2 #acneproneskin #skinc...,7411074968203953169,frekkie59,8022,319306,47786,5522,48,0,../../tiktok_data/thumbnails/74853370533836259...,"[is it really good... I'm scared, So close! Ur..."
4,7497508042016115976,1.745650e+09,cleansing @skin1004 cleanser @celimax serum @l...,7380385709847036929,cami_kana,107200,2920533,260270,13394,1895,0,../../tiktok_data/thumbnails/74975080420161159...,"[kulit mukanyaa impiann bangett, pengaruh skin..."


In [8]:
# Write the DataFrame to a new parquet file
df.to_parquet('cleaned_tiktok_data.parquet')